In [7]:
import pandas as pd
import time
from binance.client import Client
import time
from datetime import datetime


def Binance(stocks,time,start,final): #“1 Jan, 2018”
    api_key= 'ofn1WcwGN6Xkvf70Wkyv67mPr8kmfGbtOYI8Ct2roDLixKIiXwGm43Kk8YIT6Ott'
    api_secret= '0lwI0nrTC60r1TA4i4z4LD9KlaOpDogcdxVrklA7fhcobZtAXk8e6lHKfdCaanpM'
    client = Client(api_key, api_secret)
    bars= client.get_historical_klines(stocks, time,  start, final)#get_historical_klines
    date=[]
    open_hour=[]
    high_price=[]
    low_price=[]
    close_price=[]
    volume=[]
    Date_correct={'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}
    for bar in bars:
        time_=datetime.fromtimestamp(int(bar[0])//1000)#.weekday<5#.strftime(“%Y %m %d %HH:%M:%S”) +pd.Timedelta(hours=-9)
        # time_[3]=time_[3].replace(‘H’,‘’)
        # date.append(f’{int(time_[0])}-{int(time_[1])}-{int(time_[2])}-{time_[3]}’)
        date.append(time_)
        open_hour.append(float(bar[1]))
        high_price.append(float(bar[2]))
        low_price.append(float(bar[3]))
        close_price.append(float(bar[4]))
        volume.append(float(bar[5]))
    Price_data=pd.DataFrame(data={'Date':date,'Open':open_hour,'High':high_price,'Low':low_price,'Close':close_price,'Volume':volume}).set_index('Date')
    return Price_data
# valid intervals - 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
#{‘Jan’:1,‘Feb’:2,‘Mar’:3,‘Apr’:4,‘May’:5,‘Jun’:6,‘Jul’:7,‘Aug’:8,‘Sep’:9,‘Oct’:10,‘Nov’:11,‘Dec’:12}
#BTCUSDT ETHUSDT
binance=Binance('BTCUSDT','15m','1 Jan,2017','31 Dec ,2022')

In [6]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
import talib as ta

class SmaCross(Strategy):
    upper_bound = 70
    lower_bound = 30
    def init(self):
        self.rsi = self.I(ta.RSI, self.data.Close, 14)
        self.can_buy = False
        price = self.data.Close
        high_sma = 100
        low_sma = 20
        self.high = self.I(ta.SMA, price, high_sma)
        self.low = self.I(ta.SMA, price, low_sma)
    # 포지션 사이즈 조절, 손절 라인 확정.
    def next(self):
        if crossover(self.rsi, self.upper_bound):
            self.buy(sl=.005*self.data.Close[-1],tp=self.data.Close[-1]*.05+self.data.Close[-1],size=100) 
            self.can_buy = False
        elif crossover(self.lower_bound, self.rsi): 
            self.position.close()
            self.can_buy = True
        if crossover(self.low,self.high):
            self.position.close() 
                



    
bt=Backtest(binance,SmaCross,cash=1000000,commission=0.0004)
stats = bt.run()

# # stats.plot()

print(stats)

Start                     2019-09-09 02:45:00
End                       2022-12-31 09:00:00
Duration                   1209 days 06:15:00
Exposure Time [%]                   13.710053
Equity Final [$]                 1600569.4512
Equity Peak [$]                  1646870.4512
Return [%]                          60.056945
Buy & Hold Return [%]                  65.859
Return (Ann.) [%]                   15.244445
Volatility (Ann.) [%]               19.884844
Sharpe Ratio                         0.766636
Sortino Ratio                        1.693663
Calmar Ratio                         0.867249
Max. Drawdown [%]                  -17.577924
Avg. Drawdown [%]                   -1.782538
Max. Drawdown Duration      777 days 22:30:00
Avg. Drawdown Duration       13 days 20:47:00
# Trades                                  249
Win Rate [%]                        42.971888
Best Trade [%]                       4.990748
Worst Trade [%]                     -8.972289
Avg. Trade [%]                    